In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, sample_each, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train_tmp, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
if len(instances_train_tmp) > config.size:
    instances_train = np.random.choice(instances_train_tmp, config.size, replace=False)
else:
    instances_train = instances_train_tmp

In [6]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(4000, 1035)

# run

## initialize log

In [7]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [8]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 0 0 

## run

In [9]:
while epoch < config.n_epochs:
    time_start = time.time()
    sample_each(train_docs, topic_root, train=True)
    time_log = float(time.time() - time_start)
    
    sample_each(dev_docs, topic_root, train=False)
    sample_each(test_docs, topic_root, train=False)
    
#     ppl_train = get_perplexity(train_docs, topic_root)
    ppl_train=0
    ppl_dev = get_perplexity(dev_docs, topic_root)
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, ppl_dev, ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN:      VALID:       TEST: SPEC:             HIER:  \
         Time  Ep Ct    PPL         PPL         PPL     1     2     3 CHILD   
0    7.102116   0  0      0  475.837785  472.789211  0.05  0.19  0.47  0.75   
1    7.820501   1  0      0  470.640680  467.212913  0.05  0.22  0.50  0.75   
2    8.821284   2  0      0  466.410604  462.866642  0.05  0.26  0.51  0.76   
3    7.396860   3  0      0  462.757451  458.728291  0.05  0.29  0.52  0.78   
4    8.074347   4  0      0  462.877265  458.728291  0.05  0.30  0.52  0.77   
5    9.690058   5  0      0  462.307133  456.845146  0.05  0.31  0.52  0.77   
6    9.860482   6  0      0  458.775001  454.470917  0.05  0.31  0.51  0.79   
7    7.534625   7  0      0  459.787691  454.470917  0.05  0.31  0.50  0.80   
8    8.449475   8  0      0  459.270351  454.470917  0.05  0.32  0.50  0.79   
9    7.420545   9  0      0  455.924228  451.791518  0.05  0.32  0.51  0.79   
10   7.704269  10  0      0  453.694658  449.488429  0.05  0.34  0.52  0.78   
11   7.129258  11  0      0  454.200215  449.488429  0.05  0.34  0.51  0.79   
12   7.503880  12  0      0  453.767393  449.488429  0.05  0.35  0.51  0.79   
13   8.133194  13  0      0  449.422823  446.241277  0.05  0.45  0.51  0.89   
14   8.198891  14  0      0  447.180182  443.551735  0.05  0.45  0.51  0.89   
15   7.943134  15  0      0  444.762636  441.486414  0.05  0.45  0.52  0.89   
16   8.501607  16  0      0  446.207180  441.486414  0.05  0.44  0.52  0.89   
17   8.383140  17  0      0  446.952612  441.486414  0.05  0.45  0.52  0.88   
18   8.073275  18  0      0  447.473506  441.486414  0.05  0.45  0.52  0.88   
19   7.565023  19  0      0  447.168565  441.486414  0.05  0.45  0.52  0.88   
20   8.520874  20  0      0  444.520239  441.472636  0.05  0.44  0.52  0.87   
21   9.101112  21  0      0  447.986700  441.472636  0.05  0.44  0.52  0.88   
22   8.041928  22  0      0  446.273699  441.472636  0.05  0.44  0.52  0.87   
23   9.703946  23  0      0  446.465927  441.472636  0.05  0.44  0.52  0.87   
24   8.049993  24  0      0  444.833730  441.472636  0.05  0.45  0.52  0.87   
25   9.518983  25  0      0  444.429783  437.106082  0.06  0.45  0.52  0.86   
26   7.901598  26  0      0  442.141615  435.307341  0.06  0.46  0.52  0.85   
27  10.471193  27  0      0  437.067524  430.733841  0.06  0.46  0.52  0.84   
28   7.993137  28  0      0  431.823418  426.200263  0.06  0.47  0.52  0.87   
29   7.472606  29  0      0  428.457771  421.965535  0.06  0.47  0.52  0.85   
..        ...  .. ..    ...         ...         ...   ...   ...   ...   ...   
70   7.715852  70  0      0  386.785763  380.974794  0.10  0.48  0.52  0.70   
71   7.695858  71  0      0  389.398268  380.974794  0.10  0.48  0.51  0.69   
72   7.760591  72  0      0  388.208507  380.974794  0.10  0.48  0.52  0.69   
73   7.688996  73  0      0  387.823321  380.974794  0.10  0.48  0.52  0.69   
74   7.784903  74  0      0  386.488222  380.974794  0.11  0.47  0.52  0.69   
75   7.883850  75  0      0  385.227786  380.974794  0.10  0.48  0.51  0.69   
76   7.938138  76  0      0  386.381616  380.974794  0.11  0.47  0.51  0.68   
77   7.515888  77  0      0  387.743649  380.974794  0.11  0.47  0.52  0.66   
78   7.620760  78  0      0  389.897428  380.974794  0.10  0.48  0.51  0.65   
79   7.478904  79  0      0  388.951675  380.974794  0.10  0.48  0.52  0.62   
80   7.664772  80  0      0  388.771207  380.974794  0.10  0.49  0.52  0.60   
81   8.355888  81  0      0  389.041125  380.974794  0.09  0.49  0.51  0.61   
82   7.660537  82  0      0  389.829442  380.974794  0.09  0.50  0.51  0.60   
83   8.454074  83  0      0  391.086933  380.974794  0.09  0.49  0.51  0.60   
84   7.723560  84  0      0  387.999129  380.974794  0.09  0.49  0.52  0.60   
85   8.344746  85  0      0  389.643449  380.974794  0.09  0.49  0.52  0.60   
86   8.600122  86  0      0  388.423635  380.974794  0.09  0.49  0.52  0.60   
87   8.177003  87  0      0  388.048357  380.974794  0.09  0.50  0.52  

 0 4000 41783.0 ! nice ; bought price love quality & perfect recommend
   0-1 1379 8726.0 ! cover color keyboard mac bottom hard love apple blue
     0-1-2 1379 992.0 weeks description back mail problem delivery pro green received type
   0-2 1188 8736.0 carry pockets room camera comfortable pack plenty compartments hold books
     0-2-1 1188 983.0 room chargers back heavy high compartments weight briefcase bulky bigger
   0-3 398 2735.0 months started straps broke shoulder weeks strap zipper poor wheels
     0-3-1 398 386.0 targus buying replacement loved sleek zippers decent durable week lasted
   0-4 1035 6477.0 sleeve netbook protection inside power tablet & cord neoprene inch
     0-4-1 1035 774.0 protect thin larger cover -- extra built aspire usb huge
